# Vectorstores và Retrievers trong LangChain

## Giới thiệu

### Vectorstore là gì?
**Vectorstore** là database được tối ưu để lưu trữ và tìm kiếm vector embeddings. Chúng cho phép:
- **Lưu trữ** text documents dưới dạng vector representations
- **Tìm kiếm similarity** dựa trên semantic meaning
- **Scale** với millions of documents
- **Fast retrieval** với approximate nearest neighbor algorithms

### Retriever là gì?
**Retriever** là interface chuẩn trong LangChain để:
- **Tìm kiếm** relevant documents dựa trên query
- **Trả về** danh sách documents có liên quan
- **Tích hợp** dễ dàng với chains và agents
- **Customize** search strategies và parameters

### Workflow cơ bản:
1. **Documents** → **Text Splitter** → **Chunks**
2. **Chunks** → **Embedding Model** → **Vectors**
3. **Vectors** → **Vectorstore** → **Indexed Database**
4. **Query** → **Embedding** → **Similarity Search** → **Retrieved Documents**

## Setup và Import

In [ ]:
# Import các thư viện cần thiết
import os
import numpy as np
from pathlib import Path
from dotenv import load_dotenv

# LangChain components
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic

# Document processing
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# Retrievers
from langchain.retrievers import (
    ContextualCompressionRetriever,
    EnsembleRetriever
)
from langchain.retrievers.document_compressors import LLMChainExtractor

# Load environment variables
load_dotenv()

print("✓ Đã import các thư viện cần thiết")

## 1. Chuẩn bị dữ liệu mẫu

In [ ]:
# Tạo dataset mẫu về LangChain và AI
sample_documents = [
    {
        "content": """LangChain là một framework mạnh mẽ để xây dựng ứng dụng với Large Language Models (LLMs). 
        Framework này cung cấp các tools và abstractions để dễ dàng tích hợp LLMs vào applications. 
        LangChain hỗ trợ nhiều LLM providers như OpenAI, Anthropic, Cohere, và Hugging Face.""",
        "metadata": {"source": "langchain_intro", "category": "framework", "difficulty": "beginner"}
    },
    {
        "content": """Document Loaders trong LangChain giúp tải dữ liệu từ nhiều nguồn khác nhau như PDF, 
        websites, databases, và text files. Các loaders phổ biến bao gồm PyPDFLoader, WebBaseLoader, 
        CSVLoader, và TextLoader. Mỗi loader được tối ưu cho một loại data source cụ thể.""",
        "metadata": {"source": "document_loaders", "category": "data_ingestion", "difficulty": "beginner"}
    },
    {
        "content": """Text Splitters chia documents thành chunks nhỏ hơn để phù hợp với context window của LLMs. 
        RecursiveCharacterTextSplitter là lựa chọn phổ biến vì nó cố gắng preserve semantic boundaries. 
        Các tham số quan trọng bao gồm chunk_size và chunk_overlap.""",
        "metadata": {"source": "text_splitters", "category": "preprocessing", "difficulty": "intermediate"}
    },
    {
        "content": """Embeddings chuyển đổi text thành vector representations mà máy tính có thể hiểu được. 
        OpenAI Embeddings và Sentence Transformers là hai lựa chọn phổ biến. 
        Vectors này capture semantic meaning và cho phép tính toán similarity between texts.""",
        "metadata": {"source": "embeddings", "category": "vectorization", "difficulty": "intermediate"}
    },
    {
        "content": """Vector databases như FAISS, Pinecone, và Chroma được tối ưu để lưu trữ và tìm kiếm embeddings. 
        Chúng sử dụng algorithms như cosine similarity và euclidean distance để tìm nearest neighbors. 
        FAISS là lựa chọn tốt cho local development và prototyping.""",
        "metadata": {"source": "vector_databases", "category": "storage", "difficulty": "intermediate"}
    },
    {
        "content": """RAG (Retrieval-Augmented Generation) kết hợp retrieval với generation để tạo ra câu trả lời 
        chính xác hơn. Pipeline bao gồm: retrieve relevant documents, construct context, và generate response. 
        RAG giúp giảm hallucination và cung cấp thông tin up-to-date.""",
        "metadata": {"source": "rag_system", "category": "architecture", "difficulty": "advanced"}
    },
    {
        "content": """Chains trong LangChain cho phép kết nối multiple components lại với nhau để tạo complex workflows. 
        LLMChain là chain đơn giản nhất, kết hợp prompt với LLM. 
        RetrievalQA chain kết hợp retriever với LLM để answer questions dựa trên retrieved context.""",
        "metadata": {"source": "chains", "category": "orchestration", "difficulty": "intermediate"}
    },
    {
        "content": """Agents trong LangChain có khả năng reasoning và quyết định actions cần thực hiện. 
        Chúng sử dụng LLMs làm reasoning engine và có thể gọi các tools khác nhau. 
        ReAct pattern là approach phổ biến cho agent implementation.""",
        "metadata": {"source": "agents", "category": "autonomous", "difficulty": "advanced"}
    },
    {
        "content": """Memory components trong LangChain giúp lưu trữ conversation history. 
        ConversationBufferMemory lưu toàn bộ conversation, trong khi ConversationSummaryMemory 
        summarize old messages để tiết kiệm tokens. Memory quan trọng cho chatbot applications.""",
        "metadata": {"source": "memory", "category": "conversation", "difficulty": "intermediate"}
    },
    {
        "content": """Production deployment của LangChain applications cần consider factors như scalability, 
        monitoring, và cost optimization. LangSmith cung cấp observability và debugging tools. 
        Caching strategies và async processing giúp improve performance.""",
        "metadata": {"source": "production", "category": "deployment", "difficulty": "advanced"}
    }
]

# Convert thành Document objects
documents = []
for doc_data in sample_documents:
    doc = Document(
        page_content=doc_data["content"],
        metadata=doc_data["metadata"]
    )
    documents.append(doc)

print(f"✓ Đã tạo {len(documents)} documents mẫu")
print(f"\nDocument đầu tiên:")
print(f"Content: {documents[0].page_content[:150]}...")
print(f"Metadata: {documents[0].metadata}")

## 2. Text Splitting cho Documents

In [ ]:
# Split documents thành chunks nhỏ hơn
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Split all documents
all_splits = text_splitter.split_documents(documents)

print(f"Original documents: {len(documents)}")
print(f"After splitting: {len(all_splits)} chunks")
print(f"\nChunk size distribution:")

chunk_sizes = [len(chunk.page_content) for chunk in all_splits]
print(f"- Min: {min(chunk_sizes)} chars")
print(f"- Max: {max(chunk_sizes)} chars")
print(f"- Average: {np.mean(chunk_sizes):.1f} chars")

# Show some example chunks
print(f"\n=== Example Chunks ===")
for i, chunk in enumerate(all_splits[:3]):
    print(f"\nChunk {i+1}:")
    print(f"Content: {chunk.page_content}")
    print(f"Metadata: {chunk.metadata}")
    print(f"Length: {len(chunk.page_content)} chars")
    print("-" * 60)

## 3. Embedding Models - Chuyển đổi text thành vectors

In [ ]:
# Option 1: HuggingFace Embeddings (free, chạy local)
print("Khởi tạo HuggingFace Embeddings...")
hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},  # Sử dụng CPU
    encode_kwargs={'normalize_embeddings': True}  # Normalize vectors
)

# Test embedding
test_text = "LangChain là framework cho LLM applications"
test_embedding = hf_embeddings.embed_query(test_text)

print(f"✓ HuggingFace embeddings ready")
print(f"Test text: '{test_text}'")
print(f"Embedding dimension: {len(test_embedding)}")
print(f"First 5 values: {test_embedding[:5]}")

# Sử dụng HuggingFace embeddings làm default
embeddings = hf_embeddings

In [ ]:
# Option 2: OpenAI Embeddings (nếu có API key)
try:
    if os.getenv("OPENAI_API_KEY"):
        print("\nKhởi tạo OpenAI Embeddings...")
        openai_embeddings = OpenAIEmbeddings(
            model="text-embedding-3-small",  # Model mới nhất
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        
        # Test OpenAI embedding
        openai_test_embedding = openai_embeddings.embed_query(test_text)
        print(f"✓ OpenAI embeddings ready")
        print(f"OpenAI embedding dimension: {len(openai_test_embedding)}")
        
        # Có thể switch sang OpenAI nếu muốn
        # embeddings = openai_embeddings
    else:
        print("\nOpenAI API key không có - sẽ sử dụng HuggingFace")
except Exception as e:
    print(f"\nLỗi khi khởi tạo OpenAI embeddings: {e}")
    print("Tiếp tục với HuggingFace embeddings")

## 4. Tạo FAISS Vectorstore

In [ ]:
# Tạo FAISS vectorstore từ documents
print("Tạo FAISS vectorstore...")
print(f"Processing {len(all_splits)} chunks...")

# FAISS.from_documents sẽ tự động:
# 1. Embed tất cả documents
# 2. Tạo FAISS index
# 3. Store documents và metadata
vectorstore = FAISS.from_documents(
    documents=all_splits,
    embedding=embeddings
)

print(f"✓ FAISS vectorstore đã được tạo")
print(f"Total vectors: {vectorstore.index.ntotal}")
print(f"Vector dimension: {vectorstore.index.d}")

In [ ]:
# Thêm documents mới vào existing vectorstore
new_doc = Document(
    page_content="""Prompt engineering là kỹ thuật thiết kế prompts hiệu quả để tối ưu output của LLMs. 
    Các techniques bao gồm few-shot learning, chain-of-thought prompting, và instruction tuning.""",
    metadata={"source": "prompt_engineering", "category": "optimization", "difficulty": "intermediate"}
)

# Split new document
new_splits = text_splitter.split_documents([new_doc])

# Add to vectorstore
vectorstore.add_documents(new_splits)

print(f"✓ Đã thêm {len(new_splits)} chunks mới")
print(f"Total vectors hiện tại: {vectorstore.index.ntotal}")

## 5. Basic Vector Search

In [ ]:
# Test similarity search
query = "Cách sử dụng embeddings trong LangChain"

print(f"Query: '{query}'")
print("\n=== Similarity Search Results ===")

# Similarity search (default k=4)
similar_docs = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(similar_docs):
    print(f"\n--- Result {i+1} ---")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print(f"Length: {len(doc.page_content)} chars")

In [ ]:
# Similarity search với scores
query2 = "RAG system architecture"

print(f"\nQuery: '{query2}'")
print("\n=== Similarity Search with Scores ===")

# Search với similarity scores
similar_docs_with_scores = vectorstore.similarity_search_with_score(query2, k=4)

for i, (doc, score) in enumerate(similar_docs_with_scores):
    print(f"\n--- Result {i+1} (Score: {score:.4f}) ---")
    print(f"Content: {doc.page_content[:150]}...")
    print(f"Source: {doc.metadata['source']}")
    print(f"Category: {doc.metadata['category']}")

## 6. Retrievers - Interface chuẩn cho search

In [ ]:
# Tạo retriever từ vectorstore
base_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Return top 3 results
)

print("✓ Base retriever created")

# Test retriever
query3 = "text splitting strategies"
retrieved_docs = base_retriever.get_relevant_documents(query3)

print(f"\nQuery: '{query3}'")
print(f"Retrieved {len(retrieved_docs)} documents")

for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Retrieved Doc {i+1} ---")
    print(f"Content: {doc.page_content}")
    print(f"Source: {doc.metadata['source']}")

In [ ]:
# Different search types
search_types = ["similarity", "mmr"]  # Maximum Marginal Relevance

query4 = "machine learning và AI"

for search_type in search_types:
    print(f"\n=== Search Type: {search_type.upper()} ===")
    
    if search_type == "mmr":
        # MMR để giảm redundancy
        retriever = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={
                "k": 4,
                "fetch_k": 8,  # Fetch more candidates
                "lambda_mult": 0.7  # Diversity vs relevance tradeoff
            }
        )
    else:
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}
        )
    
    docs = retriever.get_relevant_documents(query4)
    
    print(f"Query: '{query4}'")
    for i, doc in enumerate(docs):
        print(f"  {i+1}. {doc.metadata['source']} - {doc.page_content[:100]}...")

## 7. Advanced Retrieval Strategies

In [ ]:
# Filtering bằng metadata
def test_metadata_filtering():
    print("=== Metadata Filtering ===")
    
    query = "LangChain framework"
    
    # Search trong beginner category only
    beginner_docs = vectorstore.similarity_search(
        query,
        k=3,
        filter={"difficulty": "beginner"}
    )
    
    print(f"Beginner level documents for '{query}':")
    for doc in beginner_docs:
        print(f"- {doc.metadata['source']}: {doc.page_content[:80]}...")
    
    # Search trong advanced category
    advanced_docs = vectorstore.similarity_search(
        query,
        k=3,
        filter={"difficulty": "advanced"}
    )
    
    print(f"\nAdvanced level documents for '{query}':")
    for doc in advanced_docs:
        print(f"- {doc.metadata['source']}: {doc.page_content[:80]}...")

test_metadata_filtering()

In [ ]:
# Score threshold filtering
def test_score_threshold():
    print("\n=== Score Threshold Filtering ===")
    
    query = "vector database performance"
    
    # Get all results with scores
    all_results = vectorstore.similarity_search_with_score(query, k=10)
    
    print(f"All results for '{query}':")
    for i, (doc, score) in enumerate(all_results):
        print(f"  {i+1}. Score: {score:.4f} - {doc.metadata['source']}")
    
    # Filter by score threshold
    threshold = 0.8
    good_results = [(doc, score) for doc, score in all_results if score <= threshold]
    
    print(f"\nResults with score <= {threshold}:")
    for doc, score in good_results:
        print(f"  Score: {score:.4f} - {doc.metadata['source']}")
        print(f"    Content: {doc.page_content[:100]}...")

test_score_threshold()

## 8. Contextual Compression Retriever

In [ ]:
# Contextual Compression để extract chỉ relevant parts
try:
    # Khởi tạo LLM cho compression
    llm = ChatAnthropic(
        model="claude-3-5-sonnet-20241022",
        temperature=0,
        anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")
    )
    
    # Tạo compressor
    compressor = LLMChainExtractor.from_llm(llm)
    
    # Tạo compression retriever
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )
    
    print("✓ Contextual Compression Retriever created")
    
    # Test compression
    query_compression = "Cách tạo embeddings trong LangChain"
    
    print(f"\nQuery: '{query_compression}'")
    print("\n=== Standard Retrieval ===")
    standard_docs = base_retriever.get_relevant_documents(query_compression)
    for i, doc in enumerate(standard_docs[:2]):
        print(f"Doc {i+1}: {doc.page_content}")
        print(f"Length: {len(doc.page_content)} chars\n")
    
    print("=== Compressed Retrieval ===")
    compressed_docs = compression_retriever.get_relevant_documents(query_compression)
    for i, doc in enumerate(compressed_docs[:2]):
        print(f"Compressed Doc {i+1}: {doc.page_content}")
        print(f"Length: {len(doc.page_content)} chars\n")
        
except Exception as e:
    print(f"Lỗi khi tạo Contextual Compression Retriever: {e}")
    print("Cần ANTHROPIC_API_KEY để sử dụng compression")

## 9. Ensemble Retriever - Kết hợp multiple retrievers

In [ ]:
# Tạo multiple retrievers với settings khác nhau
retriever1 = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

retriever2 = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 6, "lambda_mult": 0.5}
)

# Ensemble retriever kết hợp results
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever1, retriever2],
    weights=[0.6, 0.4]  # Weight cho mỗi retriever
)

print("✓ Ensemble Retriever created")

# Test ensemble
query_ensemble = "document processing pipeline"

print(f"\nQuery: '{query_ensemble}'")
print("\n=== Individual Retrievers ===")

# Results từ retriever 1
docs1 = retriever1.get_relevant_documents(query_ensemble)
print(f"Retriever 1 (Similarity):")
for i, doc in enumerate(docs1):
    print(f"  {i+1}. {doc.metadata['source']}")

# Results từ retriever 2
docs2 = retriever2.get_relevant_documents(query_ensemble)
print(f"\nRetriever 2 (MMR):")
for i, doc in enumerate(docs2):
    print(f"  {i+1}. {doc.metadata['source']}")

# Ensemble results
ensemble_docs = ensemble_retriever.get_relevant_documents(query_ensemble)
print(f"\n=== Ensemble Results ===")
for i, doc in enumerate(ensemble_docs):
    print(f"  {i+1}. {doc.metadata['source']} - {doc.page_content[:80]}...")

## 10. Save và Load Vectorstore

In [ ]:
# Save vectorstore to disk
save_path = "./langchain_vectorstore"

# Save FAISS index
vectorstore.save_local(save_path)
print(f"✓ Vectorstore saved to {save_path}")

# Load vectorstore from disk
loaded_vectorstore = FAISS.load_local(
    save_path,
    embeddings,
    allow_dangerous_deserialization=True  # Required cho FAISS
)

print(f"✓ Vectorstore loaded from {save_path}")
print(f"Loaded vectors: {loaded_vectorstore.index.ntotal}")

# Test loaded vectorstore
test_query = "LangChain applications"
loaded_results = loaded_vectorstore.similarity_search(test_query, k=2)

print(f"\nTest query on loaded vectorstore: '{test_query}'")
for i, doc in enumerate(loaded_results):
    print(f"  {i+1}. {doc.metadata['source']}: {doc.page_content[:100]}...")

## 11. Performance Analysis và Optimization

In [ ]:
import time

def benchmark_search_methods():
    """Benchmark different search methods"""
    queries = [
        "LangChain framework features",
        "vector database comparison",
        "text splitting best practices",
        "RAG implementation details",
        "embedding model selection"
    ]
    
    methods = {
        "similarity": lambda q: vectorstore.similarity_search(q, k=3),
        "similarity_with_score": lambda q: vectorstore.similarity_search_with_score(q, k=3),
        "mmr": lambda q: vectorstore.as_retriever(
            search_type="mmr", 
            search_kwargs={"k": 3, "fetch_k": 6}
        ).get_relevant_documents(q)
    }
    
    print("=== Performance Benchmark ===")
    
    for method_name, method_func in methods.items():
        start_time = time.time()
        
        for query in queries:
            results = method_func(query)
        
        end_time = time.time()
        avg_time = (end_time - start_time) / len(queries)
        
        print(f"{method_name:25s}: {avg_time:.4f}s per query")

benchmark_search_methods()

In [ ]:
# Analyze retrieval quality
def analyze_retrieval_quality():
    """Analyze quality of retrieval results"""
    test_cases = [
        {
            "query": "document loading strategies",
            "expected_sources": ["document_loaders", "text_splitters"]
        },
        {
            "query": "vector search algorithms",
            "expected_sources": ["vector_databases", "embeddings"]
        },
        {
            "query": "conversation memory management",
            "expected_sources": ["memory", "chains"]
        }
    ]
    
    print("=== Retrieval Quality Analysis ===")
    
    for test_case in test_cases:
        query = test_case["query"]
        expected = set(test_case["expected_sources"])
        
        # Get retrieval results
        results = vectorstore.similarity_search(query, k=5)
        retrieved_sources = {doc.metadata["source"] for doc in results}
        
        # Calculate metrics
        intersection = expected.intersection(retrieved_sources)
        precision = len(intersection) / len(retrieved_sources) if retrieved_sources else 0
        recall = len(intersection) / len(expected) if expected else 0
        f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        print(f"\nQuery: '{query}'")
        print(f"Expected sources: {list(expected)}")
        print(f"Retrieved sources: {list(retrieved_sources)}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1-Score: {f1_score:.2f}")

analyze_retrieval_quality()

## 12. Best Practices và Tips

In [ ]:
# Best Practice Examples
def demonstrate_best_practices():
    print("=== VECTORSTORE BEST PRACTICES ===")
    
    print("\n1. CHUNK SIZE OPTIMIZATION")
    print("   - Too small: Lack context for meaningful retrieval")
    print("   - Too large: Multiple topics, reduced precision")
    print("   - Sweet spot: 200-800 characters for general text")
    
    print("\n2. EMBEDDING MODEL SELECTION")
    print("   - HuggingFace: Free, good for prototyping")
    print("   - OpenAI: High quality, costs tokens")
    print("   - Domain-specific: Consider specialized models")
    
    print("\n3. SEARCH STRATEGY")
    print("   - Similarity: Best for exact matches")
    print("   - MMR: Better for diverse results")
    print("   - Hybrid: Combine multiple approaches")
    
    print("\n4. METADATA UTILIZATION")
    print("   - Rich metadata enables filtering")
    print("   - Source tracking for citations")
    print("   - Category/topic for domain filtering")
    
    print("\n5. PERFORMANCE OPTIMIZATION")
    print("   - Index optimization for large datasets")
    print("   - Caching for repeated queries")
    print("   - Batch processing for bulk operations")

demonstrate_best_practices()

In [ ]:
# Common pitfalls và solutions
def common_pitfalls_and_solutions():
    print("\n=== COMMON PITFALLS & SOLUTIONS ===")
    
    pitfalls = [
        {
            "problem": "Low retrieval quality",
            "causes": ["Poor chunking strategy", "Wrong embedding model", "Insufficient metadata"],
            "solutions": ["Optimize chunk size/overlap", "Try different embeddings", "Enrich metadata"]
        },
        {
            "problem": "Slow search performance",
            "causes": ["Large vector space", "Inefficient index", "Too many candidates"],
            "solutions": ["Index optimization", "Reduce embedding dimensions", "Use filtering"]
        },
        {
            "problem": "Irrelevant results",
            "causes": ["Generic embeddings", "No domain context", "Poor query formulation"],
            "solutions": ["Domain-specific embeddings", "Query expansion", "Contextual compression"]
        }
    ]
    
    for pitfall in pitfalls:
        print(f"\n❌ PROBLEM: {pitfall['problem']}")
        print(f"   Causes: {', '.join(pitfall['causes'])}")
        print(f"   ✅ Solutions: {', '.join(pitfall['solutions'])}")

common_pitfalls_and_solutions()

## 13. Cleanup

In [ ]:
# Cleanup saved files
import shutil

try:
    if Path(save_path).exists():
        shutil.rmtree(save_path)
        print(f"✓ Cleaned up {save_path}")
except Exception as e:
    print(f"Note: Could not cleanup {save_path}: {e}")

print("\n✓ Notebook execution completed!")

## Tổng kết

### **Vectorstores và Retrievers: Key Concepts**

#### **Vectorstore**
- **Purpose**: Lưu trữ và search vector embeddings
- **FAISS**: Local, fast, good cho development
- **Operations**: Add, search, save/load, filter
- **Algorithms**: Cosine similarity, Euclidean distance

#### **Embeddings**
- **HuggingFace**: Free, local, good baseline
- **OpenAI**: High quality, requires API key
- **Considerations**: Dimension, domain specificity, cost

#### **Retrievers**
- **Interface**: Chuẩn hóa search operations
- **Search Types**: Similarity, MMR, threshold
- **Advanced**: Compression, ensemble, filtering

### **Workflow Summary**
```
Documents → Split → Embed → Vectorstore → Retriever → Relevant Docs
```

### **Best Practices**
1. **Chunking**: 200-800 chars, 10-20% overlap
2. **Embeddings**: Choose based on domain và budget
3. **Search**: Start với similarity, experiment với MMR
4. **Metadata**: Rich metadata enables powerful filtering
5. **Evaluation**: Monitor precision, recall, và user satisfaction

### **Next Steps**
- **Chains**: Combine retrievers với LLMs
- **RAG Systems**: End-to-end question answering
- **Production**: Scaling, monitoring, optimization
- **Advanced**: Re-ranking, query expansion, hybrid search

Vectorstores và Retrievers là backbone của modern RAG systems!